**Question 2** 

*I. Euler's Totient Function*




In [55]:
import math
# take user input until the input is an integer  
while True:
   try:
       val = int(input("Please enter an integer: "))
       break
   except ValueError:
       print("Your input is not an integer, please try again.")
       continue
# counts the number of integers in the range of 1 and "val" that have gcd of 1 with the entered value
count=0
for i in range(0,val):
  i=i+1
  if math.gcd(i,val)==1:
    count=count+1
print('The Euler Totient Value of the integer is: ' , count)

## are we allowed to use gcd() function since it is a build in function? or are we requred to
##find gcd by implementing the Euclidian algorithm for finding gcd?

Please enter an integer: 8
The Euler Totient Value of the integer is:  4


II. Miller-Robin Algorithm



In [76]:
import random

# n is the number
# e is the error margin given by the user 
def miller_rabin(n, e):
# If number is even, it's a composite number
#The base cases for n<3 are consired below. 
  if n == 2 or n == 3:
    return True
  if n % 2 == 0:
    print('n is even')
    return False
  if n == 1:
    return False 
# first, d is found by dividing n-1 repedeatly until we are left with an odd number.
  r = 0 
  d=n-1
  while d % 2 == 0:
      r = r + 1
      d = int(d/2)
 ##witnessloop  
 # this loops runs for k times,
 #for each round we have error probablity if assigning n as a prime number falsely is
 #(1/4)^j. Hence after k rounds the toal probability of making an error is
 #((1/4))^(j^2). This should be less or equal to the error margin given by the user. 
  e=e/100
  #starting from j=1 we will increase j until to the point where pow(0.25,pow(j,2)) <= e
  # so we can use j+1 as the count of loops to guarantee an error margin less than the user enters.
  # so we use the min possible count number actually.
  j=1
  while pow(0.25,pow(j,2)) > e:
    j=j+1
  #print('j is :', j)
  for i in range(j+1):
    a = random.randint(2, n - 1);
    x = pow(a, d)%n
    if x == 1 or x == n - 1:
      continue
    for j in range(r - 1):
      x = (x * x) % n
      if x == n - 1:
        break
      else:
        return False , a
    return True




### For error to be less than 0.01% means
# 0.01% = 10^-4. Out of 1 experiment among 10000 is wrong. 
# We want error to be less than 10^-4 To achieve this;
# (1/4)^(t^2) where t is the number of different 'a' values we try, (1/4)^(t^2) < 10^-4. Therefore t must be equal or larger then 3.
# To achieve the given error margin we should try at least 3 different values of a for each n given.

In [78]:
#driver code, get n and error margin from the user
while True:
   try:
       n = int(input("Please enter an integer to check its primality: "))
       e = float(input("Please enter the error margin as integer and in the form of e%, for instance entering 0.1 means 0.1% error margin: "))
       break
   except ValueError:
       print("Your input is not an integer, please try again.")
       continue

miller_rabin(n,e)


Please enter an integer to check its primality: 9
Please enter the error margin as integer and in the form of e%, for instance entering 0.1 means 0.1% error margin: 0.01


(False, 7)

**Question 3**
Hill-Cypher Algorithm


a)[20 pts] Implement the Hill Cipher for n=2 and 3 (i.e. 2x2 and 3x3 Hill Ciphers)
 

In [43]:

import numpy as np
import random
import string
from sympy import Matrix
# All functions used for the question;

# divide_message gets input string and returns a list of arrays divides the string in the arrays with lengths of 2 or 3,
# depending on the size of the key matrix.
# each array consists of unicode code value  of the character.
# It is implicitly assumed that the input will be divisible by 2 or 3 depending on the size of the key matrix. 
# for instance; if input is string is 'DENIZE', the output will be;
# (array[(3, 4)], array[(13,8)], array[(25,4)]) 1st array corresponds to DE,2nd is NI and last is Z and E.
# when we use the ord() function with % 65, we get the desired mapping for letters and integers for Hill-Cypher Algorithm.
# If t=2 message is divided into 2 length arrays for 2x2 key matrix and if t=3 message is divided into 3 length arrays for 3x3 key matrix.  
def divide_message(messageVector, t):
  split_strings = []
  newarr=[]
  n  = 1
  for index in range(0, len(messageVector), n):
    split_strings.append(messageVector[index : index + n])
  for k in range(0, len(split_strings)):
    newarr.append((ord(split_strings[k]) % 65))
  message = np.array_split(newarr, (len(newarr))/t)
  return(message)

# this function gets the output of the divide_message, a list of the vectors of length 2 or 3, and performs
# matrix multiplication with the key matrix. The output is the cipher matrix which is the encrypted message in integer form.
# 'a' is the output of the divide_message function, key is the key matrix.
def encrypt(a,key):
  cipherMatrix = []
  for i in range(0,len(a)):
    for j in range(0,len(key)):
      cipherMatrix.append(np.dot(key[j],a[i]))

  for i in range(0, len(cipherMatrix)):
    cipherMatrix[i] = cipherMatrix[i] % 26
  return cipherMatrix

# this function is used for getting the cipher matrix in the integer form
# and return its ciphertext in the string form by using unicode. 
def letterencrypt(cipherMatrix):
  arr=[]
  for i in range(0, len(cipherMatrix)):
    arr.append(chr(cipherMatrix[i] + 65))
  separator = ''
  return separator.join(arr)

## this function is used for finding the inverse of the matrix in modulo 26.
def decrypt(key):
  ##inverse matrix of key is P(it is in double type.)
  inverse_key = Matrix(key). inv_mod(26)
  inverse_key = np.array(inverse_key) #symp to numpy
  inverse_key=inverse_key.astype(int)
  return inverse_key


In [41]:
#get the plaintext from the user
while True:
    try:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        message = input("Please enter the message you want to encrypt: ")
    except ValueError:
        print("Sorry, I didn't understand that.")
        #better try again... Return to the start of the loop
        continue
    else:
        #message was successfully parsed!
        #we're ready to exit the loop.
        break
# the operations later are done by implicitly assuming that the plaintext is all uppercase,
# so I uppercased the input message and keep in the variable 'low' whether it was given in upper case or in lowercase. 
low=False
if message.islower():
    low=message.islower()
    print(low)
    message=message.upper()
    
print('message is: ', message)


Please enter the message you want to encrypt: denizaytemiz
True
message is:  DENIZAYTEMIZ


In [44]:
# Driver code
#The below code block is for encrypting the plaintext and getting the ciphertext.
#key matrix is assumed to be 2x2
# For encryption


#divide the plaintext by 2 by 2 and store the resulting list in variable  'a'
a= divide_message(message,2)
# assuming the key matrix is given below, it is crucial that it can is invertable.
key=[(5, 2),(-7, -3)]
#we do the following operation for every element in 'a': Cipher=Key*a[i]
#cipher is the ciphermatrix
cipher = encrypt(a,key)
# Now in order to find the letter counterpart of the encryption, we use letterencrypt function and get the ciphertext.
ciphertext= letterencrypt(cipher)
print('The ciphertext for the plaintext', message,' is:')
print(ciphertext)





The ciphertext for the plaintext DENIZAYTEMIZ  is:
XTDPVHCJSOMZ


In [45]:
#Driver code
#The below code block is for decrypting the ciphertext and getting the plaintext message entered by the user back.
#the same values for ciphertext, plaintext and key matrix is used.
#For decryption

a= divide_message(ciphertext,2)

#we take the inverse of the key matrix in modulo 26
print('inverse key matrix is:')
inv_K=decrypt(key)
print(inv_K)

#this resulting matrix is the output of: decrypted=inv(key)*a[i] in modulo 26.
#the resulting is the plaintext user gives in ciphermatrix form.
decrypted = encrypt(a,inv_K)

#we get the original plaintext string by converting the cipher matrix to string form
original=letterencrypt(decrypted)
#if the original plaintext entered was in lowercase we return the plaintext as lowercase too,
#so it is the exact same plaintext.
if low:
  original=original.lower()
print('original message is:')
print(original)


inverse key matrix is:
[[ 3  2]
 [19 21]]
original message is:
denizaytemiz


In [46]:
##now the same operations will be done with 3x3 key matrix:

#get the plaintext from the user
while True:
    try:
        # Note: Python 2.x users should use raw_input, the equivalent of 3.x's input
        message = input("Please enter the message you want to encrypt: ")
    except ValueError:
        print("Sorry, I didn't understand that.")
        #better try again... Return to the start of the loop
        continue
    else:
        #message was successfully parsed!
        #we're ready to exit the loop.
        break
# the operations later are done by implicitly assuming that the plaintext is all uppercase,
# so I uppercased the input message and keep in the variable 'low' whether it was given in upper case or in lowercase. 
low=False
if message.islower():
    low=message.islower()
    print(low)
    message=message.upper()
    
print('message is: ', message)


Please enter the message you want to encrypt: denizaytemiz
True
message is:  DENIZAYTEMIZ


In [47]:
# Driver code
#The below code block is for encrypting the plaintext and getting the ciphertext.
#key matrix is assumed to be 3x3
# For encryption


#divide the plaintext by 3 by 3 and store the resulting list in variable  'a'
a= divide_message(message,3)
# assuming the key matrix is given below, it is crucial that it can is invertable.
key=[(1, 2, 3),(0, 1,4),(5,6,0)]
#we do the following operation for every element in 'a': Cipher=Key*a[i]
#cipher is the ciphermatrix
cipher = encrypt(a,key)
# Now in order to find the letter counterpart of the encryption, we use letterencrypt function and get the ciphertext.
ciphertext= letterencrypt(cipher)
print('The ciphertext for the plaintext', message,' is:')
print(ciphertext)




The ciphertext for the plaintext DENIZAYTEMIZ  is:
YENGZIWJAZEE


In [48]:
#Driver code
#The below code block is for decrypting the ciphertext and getting the plaintext message entered by the user back.
#the same values for ciphertext, plaintext and key matrix is used.
#For decryption

a= divide_message(ciphertext,3)

#we take the inverse of the key matrix in modulo 26
print('inverse key matrix is:')
inv_K=decrypt(key)
print(inv_K)

#this resulting matrix is the output of: decrypted=inv(key)*a[i] in modulo 26.
#the resulting is the plaintext user gives in ciphermatrix form.
decrypted = encrypt(a,inv_K)

#we get the original plaintext string by converting the cipher matrix to string form
original=letterencrypt(decrypted)
#if the original plaintext entered was in lowercase we return the plaintext as lowercase too,
#so it is the exact same plaintext.
if low:
  original=original.lower()
print('original message is:')
print(original)


inverse key matrix is:
[[ 2 18  5]
 [20 11 22]
 [21  4  1]]
original message is:
denizaytemiz


b)Create software that can perform a fast known plaintext attack on a Hill cipher, given the dimension n of the Hill Cipher. How fast are your algorithms, as a function of n?


In [49]:
## plain text attack for 2x2  key matrix
import math
#we divide the plaintext message to 2 -lengthed vectors to do matrix multiplication later on.

# We know K*P=C, where K is key matrix, P is plaintext and C is ciphertext.
# invertablematrix function creates a 2x2  plaintext matrix that is invertable. 
# for instance; assuming plain text=howaand we are using 2x2 key matrix, the corresponding plaintext matrix will be
# [(7,22),(14,0)] corresponding to [(h,w),(o,a)]
# Key matrix will be found by K=C*inv(P) (mod 26). We try to figure out a suitable P for this operation.
# For P matrix to be invertable its determinant should be relatively prime with 26.
#In this function we concatenate any 2 length vectors of plaintext with
# other parts of the plaintext message until we obtain a 2x2 invertable matrix. The invertable matrix and the index's of the 
#vectors are outputted.
def invertablematrix1(plaintext):
  P=np.array(divide_message(plaintext,2))
  l=len(P)
  for i in range(0,l):
    for j in range(1,l):
      new_P = np.concatenate((P[i],P[j]),axis=0)
      new_P = new_P.reshape(2,2)
      new_P = np.transpose(new_P)
      print('')
      print(new_P)
      print('')
      det=round(np.linalg.det(new_P))
      print('determinant :',det)
    if math.gcd(det,26) == 1:
      print('this is invertable. ')
     # print(new_P)
      return new_P, i, j
    else:
      print('this is not invertable')
# The keyMatrix function gets the invertable matrix(B) which is the plaintext matrix and the i and j as the inputs.
# The  respective cipher matrix C corresponding to the plaintext matrix B is created and
# by taking the inverse of B, the following operation is being done:
# A=C*inv(B)in mod 26. Since we know inv(B) and C, we can figure out the key matrix(A)
def keyMatrix1(B,i, j):
  C=np.array(divide_message(ciphertext,2))
  new_C = np.concatenate((C[i],C[j]),axis=0)
  new_C = new_C.reshape(2,2)
  C = np.transpose(new_C)
  print('corresponding cipher matrix is:')
  print(C)
  invB=decrypt(B)
  A = np.matmul(C,invB)
  A = A%26
  print('the key matrix:')
  return A

In [9]:
#Driver code
#for 2x2 key matrix
#In the known plaintext attacks; we assume that we know the partial of plaintext, and the corresponding ciphertext and the size of the key matrix.
#Assume that the ciphertext and part of the plaintext we know are as below. 
import time
ciphertext='ZWSENIUSPLJVEULNVP'
plaintext='HOWAREYOUTOD'
# We  gonna try to find the key matrix and decrypt the ciphertext message, in order to get
# the full plaintext message.
# variables tic&toc is for keeping the time of execution.
tic = time.perf_counter()
#we find the invertable plaintext matrix and the corresponding indeces
B, i, j = invertablematrix1(plaintext)
#Now using the keyMatrix function, it is possible to determine the key matrix used for the encryption.
print('')
print('The key matrix is:')
key=keyMatrix1(B,i, j)
print(key)
#Driver code
#For decryption
#Since we know the key matrix we can use decryption on the ciphertext as we did in the part a of the question.
#And get the full of the plaintext.

print('The integer counterpart of the ciphertext is given:')
print(divide_message(ciphertext,2))
a= divide_message(plaintext,2)
print('')
print('Lets find the inverse of the 2x2 key matrix by decrypt function: ')
inv_K=decrypt(key)
print(inv_K)
print('')
print('Now,we decrypt the ciphertext with  multiplying the inverse key matrix and the cipher matrix.')
print('')
print('The output of the multiplication is as follows:')
C=divide_message(ciphertext,2)
print(encrypt(C,inv_K))
print('')
print('Now in order to find the letter counterpart of the encryption(unicode), we use letterencrypt function:')
decrypted = encrypt(C,inv_K)
print('')
print(decrypted)
print('')
print('The full version of the plaintext message is:')
print(letterencrypt(decrypted))
toc =time.perf_counter()
print('The time it took to execute the code for 2x2 key matrix is:')
print(toc-tic)


[[ 7 22]
 [14  0]]

determinant : -308

[[ 7 17]
 [14  4]]

determinant : -210

[[ 7 24]
 [14 14]]

determinant : -238

[[ 7 20]
 [14 19]]

determinant : -147

[[ 7 14]
 [14  3]]

determinant : -175
this is invertable. 

The key matrix is:
corresponding cipher matrix is:
[[25  9]
 [22 21]]
the key matrix:
[[15 11]
 [12  3]]
The integer counterpart of the ciphertext is given:
[array([25, 22]), array([18,  4]), array([13,  8]), array([20, 18]), array([15, 11]), array([ 9, 21]), array([ 4, 20]), array([11, 13]), array([21, 15])]

Lets find the inverse of the 2x2 key matrix by decrypt function: 
[[17  7]
 [10  7]]

Now,we decrypt the ciphertext with  multiplying the inverse key matrix and the cipher matrix.

The output of the multiplication is as follows:
[7, 14, 22, 0, 17, 4, 24, 14, 20, 19, 14, 3, 0, 24, 18, 19, 20, 3]

Now in order to find the letter counterpart of the encryption(unicode), we use letterencrypt function:

[7, 14, 22, 0, 17, 4, 24, 14, 20, 19, 14, 3, 0, 24, 18, 19, 20, 3

In [52]:
## plain text attack for 3x3  key matrix
import math
#we divide the plaintext message to 3 -lengthed vectors to do matrix multiplication later on.

# We know K*P=C, where K is key matrix, P is plaintext and C is ciphertext.
# invertablematrix function creates a 3x3  plaintext matrix that is invertable. 
# for instance; assuming plain text=howaand we are using 3x3 key matrix, the corresponding plaintext matrix will be
# Key matrix will be found by K=C*inv(P) (mod 26). We try to figure out a suitable P for this operation.
# For P matrix to be invertable its determinant should be relatively prime with 26.
#In this function we concatenate any 3 length vectors of plaintext with
# other parts of the plaintext message until we obtain a 3x3 invertable matrix. The invertable matrix and the index's of the 
#vectors are outputted.
def invertablematrix2(plaintext):
  P=np.array(divide_message(plaintext,3))
  l=len(P)
  for i in range(0,l):
    for j in range(1,l):
      for m in range(2,l):
        new_P = np.concatenate((P[i],P[j],P[m]),axis=0)
        new_P = new_P.reshape(3,3)
        new_P = np.transpose(new_P)
        print('')
        print(new_P)
        print('')
        det=round(np.linalg.det(new_P))
        print('determinant :',det)
      if math.gcd(det,26) == 1:
        print('this is invertable. ')
        # print(new_P)
        return new_P, i, j,m
      else:
        print('this is not invertable')
# The keyMatrix function gets the invertable matrix(B) which is the plaintext matrix and the i and j as the inputs.
# The  respective cipher matrix C corresponding to the plaintext matrix B is created and
# by taking the inverse of B, the following operation is being done:
# A=C*inv(B)in mod 26. Since we know inv(B) and C, we can figure out the key matrix(A)
def keyMatrix2(B,i, j,m):
  C=np.array(divide_message(ciphertext,3))
  new_C = np.concatenate((C[i],C[j],C[m]),axis=0)
  new_C = new_C.reshape(3,3)
  C = np.transpose(new_C)
  print('corresponding cipher matrix is:')
  print(C)
  invB=decrypt(B)
  A = np.matmul(C,invB)
  A = A%26
  print('the key matrix:')
  return A

In [54]:
#Driver code
#for 3x3 key matrix
#In the known plaintext attacks; we assume that we know the partial of plaintext, and the corresponding ciphertext and the size of the key matrix.
#Assume that the ciphertext and part of the plaintext we know are as below. 
import time
ciphertext='ZWSENIUSPLJVEULNVP'
plaintext='HOWAREYOUTOD'
# We  gonna try to find the key matrix and decrypt the ciphertext message, in order to get
# the full plaintext message.
# variables tic&toc is for keeping the time of execution.
tic = time.perf_counter()
#we find the invertable plaintext matrix and the corresponding indeces
B, i, j,m = invertablematrix2(plaintext)
#Now using the keyMatrix function, it is possible to determine the key matrix used for the encryption.
print('')
print('The key matrix is:')
key=keyMatrix2(B,i, j,m)
print(key)

#Driver code
#For decryption
#Since we know the key matrix we can use decryption on the ciphertext as we did in the part a of the question.
#And get the full of the plaintext.

print('The integer counterpart of the ciphertext is given:')
print(divide_message(ciphertext,3))
a= divide_message(plaintext,3)
print('')
print('Lets find the inverse of the 3x3 key matrix by decrypt function: ')
inv_K=decrypt(key)
print(inv_K)
print('')
print('Now,we decrypt the ciphertext with  multiplying the inverse key matrix and the cipher matrix.')
print('')
print('The output of the multiplication is as follows:')
C=divide_message(ciphertext,3)
print(encrypt(C,inv_K))
print('')
print('Now in order to find the letter counterpart of the encryption(unicode), we use letterencrypt function:')
decrypted = encrypt(C,inv_K)
print('')
print(decrypted)
print('')
print('The full version of the plaintext message is:')
print(letterencrypt(decrypted))
toc =time.perf_counter()
print('The time it took to execute the code for 3x3 key matrix is:')
print(toc-tic)


[[ 7  0 24]
 [14 17 14]
 [22  4 20]]

determinant : -5644

[[ 7  0 19]
 [14 17 14]
 [22  4  3]]

determinant : -6077
this is invertable. 

The key matrix is:
corresponding cipher matrix is:
[[25  4 11]
 [22 13  9]
 [18  8 21]]
the key matrix:
[[17  8  6]
 [10 19  7]
 [ 2 18 23]]
The integer counterpart of the ciphertext is given:
[array([25, 22, 18]), array([ 4, 13,  8]), array([20, 18, 15]), array([11,  9, 21]), array([ 4, 20, 11]), array([13, 21, 15])]

Lets find the inverse of the 3x3 key matrix by decrypt function: 
[[ 3 20 18]
 [24  7 21]
 [16 12 25]]

Now,we decrypt the ciphertext with  multiplying the inverse key matrix and the cipher matrix.

The output of the multiplication is as follows:
[7, 14, 22, 0, 17, 4, 14, 11, 1, 19, 14, 3, 12, 25, 7, 1, 20, 3]

Now in order to find the letter counterpart of the encryption(unicode), we use letterencrypt function:

[7, 14, 22, 0, 17, 4, 14, 11, 1, 19, 14, 3, 12, 25, 7, 1, 20, 3]

The full version of the plaintext message is:
HOWAREOLBT

I believe the reason I could not recover the plaintext completely true is that, the key matrix we found is not used for encryting this message. However it still satifies the initial matrix multiplication. As you see the recovered parts from the message HOWARE and TOD is correct. Which means there are multiple K's for satisfying C=K*P, and the one we select in particular was not the one that is used for encryption.

